## Insertando un documento

Usa alguna de las formas de conectarse a la DB de Mongo Atlas(nube) en este caso desde VSCode. Creamos una nueva carpeta *04-insert-doc* y dentro un archivo .mongodb

Vamos a crear una nueva BD. No es necesario crearla explicitamente, simplemente si le decimos vamos a usar tal base de datos y esta no esta creada, el inmediatamtente la crea. Lo mismo para las colecciones.

La primera instruccion para insertar un documento es *insertOne*

    use("platzi_store")
    db.products.insertOne({
        name: "Licuadora",
        price: 1100
    })

    db.products.insertOne({
        name: "Microondas",
        price: 360
    })

Al correr el documento, sale:

![](https://i.imgur.com/McWuZYY.png)

Como si solo hubiera insertado uno solo,SIgnigica el *playground* que estamos corriendo se queda solo con la ultima instruccion.

Pero si revisamos encontraremos efectivamente estan los dos docs. 

    db.products.find().count()

Al intentar usar el comando *show collections* en VScode no lo deja hacer, pero desde la *MongoSh* si.

    show collections


Observa esta consulta:

    [
    {
        "_id": {
        "$oid": "640b9e7fc3aa787c39b067be"
        },
        "name": "Licuadora",
        "price": 1100
    },
    {
        "_id": {
        "$oid": "640b9e7fc3aa787c39b067bf"
        },
        "name": "Microondas",
        "price": 360
    }
    ]

*oid* es object id y es unico y asignado directamete por mongo, igualmente tu puedes asignar tus propios IDs, pero ten cuidado.

    db.products.insertOne({
        _id: 1,
        name: "tostadora",
        price: "341"    
    })

## Insertando varios documentos

Crearemos una nueva carpeta llamada *05-insert-many* y dentro el archivo *products.mongodb*. Usar la siguiente funcion con cuidado para borrar todos los documentos de una coleccion:

    use("platzi_store")
    db.products.drop()

Al parecer como esta BD de datos solo contenia una coleccion cuyos documentos fueron borrados en su totalidad, al consultar con el comando *show dbs* desde mongo shell, pues *platzi_store* tambien fue borrada. Igualmente este no es un problema por ahora.

Al usar la instruccion *insertMany* se usa la estructura de un array:

        db.products.insertMany([{
            name: "ducha",
            price: 540
        },
        {
            name: "horno",
            price: 1200
        },
        {
            name: "nevera",
            price: 980
        },
        ])


### ¿QUe pasa si tenemos ids duplicados?

volvamos a realizar el ejercicio esta vez asignando manualmente los IDs.

    db.products.insertMany([{
        _id : 1,
        name: "ducha",
        price: 540
    },
    {
        _id : 2,
        name: "horno",
        price: 1200
    },
    {
        _id : 1,
        name: "nevera",
        price: 980
    },
    {
        _id : 4,
        name: "plancha",
        price: 120
    },
    ])


En este caso, saldra un error de duplicacion, sin embargo insertara los dos primeros documentos. ¿COmo podriamos entonces dejar solo por fuera los que tienen error de duplicacion, pero los demas no?

Con la instruccion *ordered=false*. Pero a mi no me funciona

## Actualizando un documento

### Desde Mongo Atlas

Directamente desde la interfax grafica

### Con Comandos

Crear una nueva carpeta *06-update-doc* y dentro el respectivo playground

#### updateOne

Se debe hacer uso de los *operadores atomicos*. Por ejemplo, si queremos cambiar el nombre del documento con ID = 2, el *atomic operator* es *$set*.

La primera parte corresponde al *query* y la segunda parte a la respectiva actualizacion.

    db.products.updateOne(
    // query
    {
        _id:2},
    {
        /**
         * field: The field name
         * expression: The expression.
         */
        $set: {
          name: "horno magico"
        }
    }
    )

##### Ejemplo. 

Actualizar el documento correspondiente al de la nevera, y cambiar su nombre y precio:

    use("platzi_store")
    
    db.products.updateOne(
        // query
        {
            name:"nevera"},
        {

            $set: {
            name: "nevera magica",
            price: 1234
            }
        }
    )

##### Ejemplo2.

Actualizar el documento correspondiente a la ducha, y cambiar nombre y precio, y ademas agregar un nuevo atributo llamado tags.

    db.products.updateOne(
        {
            name:"ducha"
        },
        {
            $set: {
            name: "ducha magica",
            price: 56789,
            tags: ['A', 'B', 'C'],
            }
        }
    )

#### Operador $inc

al igual que $set, otro operador es $inc que es para incrementar atributos numericos. El valor actual mas el incremento. Tambien se pueden incrementos negativos.

    db.products.updateOne(
    {
        _id:4
    },
    {

        $inc: {
          price: 240,
        }
    })


#### Funcion ObjectId

Para los IDs generados autogenerados, se necesita dicha funcion para el query. 

    db.products.updateOne(
    {
        _id: ObjectId("640e4b1fb946da9d6a2d47e9")
    },
    {
        $inc: {
            price:1000,
        }
    }

    )

## Actualizando varios documentos

Creamos una nueva carpeta 07-update-many. Nos conectaremos a la base de datos *sample training* e incrementaremos la poblacion de todos los registros que coinciden con Cleveland en la coleccion *zips*

Recuerda en MongoSH puedes usar el comando *show collections* para mostrar todas las colecciones.

    show collections
    db.zips.find().count()

Hay 29430 documentos en toda la coleccion. Ahora los que coinciden con *CLEVELAND*

    db.zips.find( {city:"CLEVELAND" } ).count()

Solo hay 34.

### updateMany

La primera parte como se hizo anteriormente es el *query*.

    use("sample_training")

    db.zips.updateMany(
    {
        city:"CLEVELAND" 
    },

    {

        $inc: {
          pop: 1000
        }
    }
    )

Aqui observamos el resultado de la ejecucion sobre los 34 documentos

![](https://i.imgur.com/QSiM0kH.png)

#### Ejemplo.

Usar el operador *set* para crear un nuevo atributo llamado *myAtributo*

    
    db.zips.updateMany(
    {
        city:"CLEVELAND"
    },

    {
        $set: {
          myAtributo: "hola"
        }
    }
    
    )

Si hacemos nuevamente una query, encontraremos que se añadio un nuevo atributo

    db.zips.find( {city:"CLEVELAND" } )

#### ¿Que pasara si queremos renombra el atributo creado anteriormente?

Usamos el operador *$rename* y no cambiaria el valor

    db.zips.updateMany(
    {
        city:"CLEVELAND"
    },

    {
        $rename: {
          myAtributo: "my_data"
        }
    }
    )

Si ahora lo que queremos es eliminar el atributo que acabamos de crear, se usa *$unset*

    db.zips.updateMany(
    {
        city:"CLEVELAND"
    },

    {
        $unset: {
          my_data: ""
        }
    }
    )

## Array Update Operators

Copiar la informacion del siguiente enlace:

https://gist.github.com/nicobytes/fbd8c63977217855ba8afd3e240651c9

### Operador push

La nueva coleccion *inventory* contiene un atributo llamado *tags* que es un array.

    use("platzi_store")

    db.inventory.updateOne(
        {
            _id: 4
        },
        {

            $push: {
            tags: "headphone"
            }
        }
    )

    db.inventory.find({_id:4})

### Operador pull

Quita elementos de los arrays. Removeremos del arreglo tags el elemento *book* 

Como no hay ningun *query* va a traer todos los documentos que encuentre, y a continuacion realizara el operador de *pull*.

    db.inventory.updateMany({},
    {

        $pull: {
        tags: 'book'
        }
    })

El resultado muestra que hizo un match de 5 elementos, y solo modifico 3.

![](https://i.imgur.com/edSUOmi.png)


### Operador in

Ahora, ¿Que tal si removemos dos o mas tags a la vez?

En este caso usamos otro operador llamado *$in*. Eliminaremos *apliance* y *school*

    db.inventory.updateMany({},
    {

        $pull: {
        tags: {
            $in: ["appliance", "school"]
        }
        }
    })


Tambien se puede usar el mismo operador para realizar consultas, en este caso encontrar documentos que tengan el tag *bag*:

     db.inventory.find({tags: {$in: ["bag"] }})



## Update or Insert

El dataset sera informacion de sensores, donde por cada sensor y por cada fecha se guarda un documento.

Si quisieramos agregar una lectura del *sensor 1* para una fecha ya existente, podriamos agregarla simplemente asi:

    use("platzi_store")

    db.iot.updateOne({
        sensor: 'A001',
        date: '2022-01-01'},
        {
            $push: {
            readings: 201
            }
        }
    )

Y tendriamos una nueva lectura.

Si ahora quisieramos agregar dos lecturas consecutivas, esta vez al sensor 2, y en una nueva fecha. Primero tendriamos que hacer un set, y luego un push:

    use("platzi_store")

    db.iot.insertOne(
    {
        sensor: 'A002',
        date: '2022-01-03',
        readings: [5]
    }
    )


Para la segunda lectura, cometeremos un error a proposito: 

    db.iot.updateOne(
    {
        sensor: 'A002',
        date: '2022-01-03',    
    },

    {
        $push: {
        readings: [1013]
        }
    }
    )

En el caso anterior, en vez de agregar un valor, agregamos fue un arreglo:

    _id: ObjectId("640eae58e6b4515a323afeaf"),
    sensor: 'A002',
    date: '2022-01-03',
    readings: [ 5, [ 1013 ] ]


### operador pop

para borrar el ultimo registro de un arreglo, usaremos pop con valor de 1, y valor de -1 para borrar el ultimo y primero respectivamente.

    use("platzi_store")

    db.iot.updateOne(
    {
        sensor: 'A002',
        date: '2022-01-03',    
    },

    {
        $pop: {
        readings: 1,
        }
    }
    )




Finalmente agregaremos la segunda lectura, esta vez de forma correcta:

    db.iot.updateOne(
    {
        sensor: 'A002',
        date: '2022-01-03',    
    },

    {
        $push: {
        readings: 1013
        }
    }
    )

Y aqui el resultado:

    _id: ObjectId("640eb1b5b6468294928d3d30"),
    sensor: 'A002',
    date: '2022-01-03',
    readings: [ 5, 1013 ]


Sin embargo hay una manera mas facil para no preocuparse si es una insercion o actualizacion, y para eso viene el *upsert*

### upsert

Que es la fusion entre update e insert, y agregamos el statement. Para el siguiente caso, como ya existe un documento, lo que hace es actualizar la informacion.

    db.iot.updateOne(
    {
        sensor: 'A002',
        date: '2022-01-03',    
    },

    {
        $push: {
        readings: 2023
        }
    },

    {
        upsert:true
    }
    )



## Eliminando documentos

Copiaremos el dataset de aqui:

    https://gist.github.com/nicobytes/fbd8c63977217855ba8afd3e240651c9

Crearemos un nuevo folder: *10-delete*


Borraremos un registro

    db.products.deleteOne( {_id: 1} )

Ahora ejecutaremso un deleteMany, por ejemplo borrar todos aquellos docs que tengan el precio en 100 dolares. 

    db.products.deleteMany(
        { price: 100}
    )

Y finalmente para borrar todos los docs de una coleccion con *drop*

    db.products.drop()